In [1]:
import torch

from libsimplesn import SimpleSN


simplesn = SimpleSN(survey='pantheon-g10')

In [2]:
zcmb = torch.load(simplesn._surveydir / f'{simplesn.survey}-zcmb.pt')
vars = torch.load(simplesn._surveydir / f'{simplesn.survey}-vars.pt')

N = len(zcmb)

for simplesn.N in (
    1_000, 2_000, 5_000,
    10_000, 20_000, 50_000,
    100_000,
    # 200_000, 500_000, 1_000_000
):
    for simplesn.suffix in range(0, 1):
        idx = (
            torch.randint(N, size=(simplesn.N,)) if simplesn.N > N else
            torch.randperm(N)[:simplesn.N] if simplesn.N < N else
            torch.arange(N, dtype=torch.long)
        )

        simplesn.zcmb = zcmb[idx].cpu()
        simplesn.vars_scale_tril = torch.linalg.cholesky(vars[idx]).cpu()

        for cfgname, simplesn.datatype in (
            ('simplesn.yaml', 'specz'),
            ('simplesn.yaml', 'photoz'),
            ('simplesn-marginal.yaml', 'mphotoz'),
        ):
            config = simplesn.config(cfgname, gen=True)

            for simplesn.version in range(10):
                simplesn.trace = trace = config.mock()
                simplesn.data = {
                    key: val.cpu() if torch.is_tensor(val) else val
                    for key, val in trace.nodes.items()
                    for val in [val['value']]}